In [1]:
%cd ..

/mnt/e/projects/face_recognition


In [2]:
import os

import albumentations as A
import cv2
import pandas as pd
import torch
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
from torchvision.models import efficientnet_v2_s
from sklearn.model_selection import train_test_split

from config import CFG

In [3]:
def get_dataset_and_filter_by_label_trashold(label_trashold: int = 10):
    """
    удалим из датасета, тех людей, у которых мало фото - label_trashold.
    Увеличивая label_trashold, мы умешьнаем размер датасета.

    """
    df_identity = pd.read_csv(CFG.identity_path, sep=" ", header=None).sort_values(by=0).reset_index(drop=True)
    df_identity.columns = ["image", "label"]
    cropped_imgs = os.listdir(CFG.img_folder_dst)
    data = pd.DataFrame({"image": cropped_imgs})
    data = data.join(df_identity.set_index("image"), on="image", how="left")
    data_tmp = data.groupby(["label"]).agg({"label": "count"}).rename(columns={"label": "label_count"})
    print(f"Число уникальных людей {len(data_tmp)}, каждый человек имеет, как минимум {label_trashold} фоток.")

    del_label = data_tmp.loc[data_tmp.label_count <= label_trashold, :].index.values
    mask = data["label"].isin(set(del_label))
    data = data[~mask].reset_index(drop=True)
    print(f"Всего фоток {len(data)}")
    df_labels = data["label"].unique()
    map_lables = {l: i for i, l in enumerate(df_labels)}
    data["label"] = data["label"].map(lambda x: map_lables[x])

    return data


def split_dataset_by_photo(df, label_col, num_val_samples_per_class):
    validation_data = []
    train_data = []
    test_data = []

    for label in df[label_col].unique():
        label_data = df[df[label_col] == label]

        val_samples = label_data.sample(num_val_samples_per_class)
        validation_data.append(val_samples)
        label_data = label_data.drop(val_samples.index)

        test_samples = label_data.sample(num_val_samples_per_class)
        test_data.append(test_samples)
        label_data = label_data.drop(test_samples.index)

        train_data.append(label_data)

    train_df = pd.concat(train_data).reset_index(drop=True)
    validation_df = pd.concat(validation_data).reset_index(drop=True)
    test_df = pd.concat(test_data).reset_index(drop=True)
    print(f"df({len(df)}) -> train({len(train_df)}) val({len(validation_df)}) test({len(test_df)})")

    return train_df, validation_df, test_df


# Split the dataset

# df_train, df_val, df_test = split_by_person()
df_train, df_val, df_test = split_dataset_by_photo(
    get_dataset_and_filter_by_label_trashold(CFG.label_trashold),
    "label",
    4,
)

Число уникальных людей 10174, каждый человек имеет, как минимум 28 фоток.
Всего фоток 59959
df(59959) -> train(43767) val(8096) test(8096)


In [4]:
class CelebaDataet(Dataset):
    def __init__(self, df: pd.DataFrame) -> None:
        # image, label
        self.df = df.values
        self.transform = A.Compose(
            [
                # A.Resize(height=128, width=128),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2(),
            ]
        )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path, label = self.df[index]
        img = cv2.imread(os.path.join(CFG.img_folder_dst, img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image=img)["image"]
        return img, label

In [5]:
train_dataset = CelebaDataet(df_train)
val_dataset = CelebaDataet(df_val)
test_dataset = CelebaDataet(df_test)

train_dataloader = DataLoader(train_dataset, batch_size=26, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=26, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=26, shuffle=False)

In [6]:
labels_amount = len(df_train["label"].unique())
labels_amount

2024

In [7]:
model = efficientnet_v2_s()
model.load_state_dict(torch.load("models/efficientnet_v2_s.pth"))
model.classifier = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(in_features=1280, out_features=labels_amount))
None

In [8]:
from copy import deepcopy
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train(model, train_loader, val_loader, opt, epochs=10):
    train_losses, val_losses, val_full_acc, train_full_acc = [], [], [], []
    best_acc = 0.0
    loss_fn = nn.CrossEntropyLoss()
    best_model_weights = deepcopy(model.state_dict())

    for epoch in range(epochs):
        print(f"========= Epoch {epoch + 1} / {epochs} =========")

        # TRAIN
        model.train()
        current_train_loss = 0
        current_train_correct = 0

        for inputs, labels in tqdm(train_loader):
            X_batch = inputs.to(device)
            Y_batch = labels.to(device)

            opt.zero_grad()

            # forward
            Y_pred = model(X_batch)
            preds = torch.argmax(Y_pred, 1)
            loss = loss_fn(Y_pred, Y_batch)
            loss.backward()
            opt.step()

            current_train_loss += loss.item() * X_batch.size(0)
            current_train_correct += torch.sum(preds == Y_batch)

        opt.step()

        train_loss = current_train_loss / len(train_dataset)
        train_losses.append(train_loss)
        train_acc = current_train_correct / len(train_dataset)
        train_full_acc.append(train_acc)
        print("train loss =", train_loss)
        print("train acc = {:.2f}%".format(train_acc.item() * 100))

        # VALIDATION
        model.eval()
        current_val_loss = 0
        current_val_correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                X_val = inputs.to(device)
                Y_val = labels.to(device)

                outputs = model(X_val)
                val_loss = loss_fn(outputs, Y_val)
                preds = torch.argmax(outputs, 1)
                current_val_correct += torch.sum(preds == Y_val)
                current_val_loss += val_loss.item() * X_val.size(0)

        val_acc = current_val_correct / len(val_dataset)
        val_loss = current_val_loss / len(val_dataset)

        print("val loss =", val_loss)
        print(f"val acc = {val_acc.item() * 100:.2f}%")
        val_losses.append(val_loss)
        val_full_acc.append(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_weights = deepcopy(model.state_dict())
            print("Save new model!")

    return best_model_weights, train_losses, val_losses, val_full_acc, train_full_acc

In [9]:
model.to(device)

opt = torch.optim.AdamW(model.parameters())
best_model_weights, train_losses, val_losses, val_full_acc, train_full_acc = train(
    model, train_dataloader, val_dataloader, opt, epochs=10
)

========= Epoch 1 / 10 =========


100%|██████████| 1684/1684 [15:07<00:00,  1.86it/s]


train loss = 7.0746523576594775
train acc = 0.78%
val loss = 6.094135032340943
val acc = 2.67%
Save new model!
========= Epoch 2 / 10 =========


100%|██████████| 1684/1684 [14:58<00:00,  1.87it/s]


train loss = 4.983188976686857
train acc = 11.32%
val loss = 4.228897669687573
val acc = 19.37%
Save new model!
========= Epoch 3 / 10 =========


...

train loss = 0.37747268182873195
train acc = 90.98%
val loss = 1.1824011374356984
val acc = 76.47%
Save new model!
========= Epoch 9 / 10 =========


100%|██████████| 1684/1684 [11:21<00:00,  2.47it/s]


train loss = 0.2922997477205177
train acc = 92.99%
val loss = 1.151382854988747
val acc = 78.48%
Save new model!
========= Epoch 10 / 10 =========


100%|██████████| 1684/1684 [12:53<00:00,  2.18it/s]


train loss = 0.24257110494495618
train acc = 93.98%
val loss = 1.1887537687059802
val acc = 77.17%


In [12]:
torch.save(best_model_weights, "models/efficientnet_v2_s_ce_10_epoch.pth")

In [15]:
print(labels_amount)

2024


In [14]:
def test_result(model, test_loader):
    model.eval()
    with torch.no_grad():
        current_test_acc = 0
        for inputs, labels in tqdm(test_loader):
            X_test = inputs.to(device)
            Y_test = labels.to(device)
            outputs = model(X_test)
            preds = torch.argmax(outputs, 1)
            current_test_acc += torch.sum(Y_test == preds)
    print("Correct answers: {} from {} ".format(current_test_acc, len(test_dataset)))
    test_acc = current_test_acc / len(test_dataset)
    print("Test accuracy = {:.2f}%".format(test_acc * 100))


best_model = efficientnet_v2_s()
best_model.classifier = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(in_features=1280, out_features=labels_amount))
best_model.load_state_dict(best_model_weights)

test_result(best_model.to(device), test_dataloader)

100%|██████████| 312/312 [01:54<00:00,  2.72it/s]


Correct answers: 6325 from 8096 
Test accuracy = 78.13%
